In [1]:
import re
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
# Reading the text file:-
with open('65027 XII.txt') as file:
    reader = file.read()

In [3]:
# Defining the patterns to get 'Date', 'School Code', 'School Name' and Region' :-
date_pattern = r"DATE:- (\d{2}/\d{2}/\d{4})"
school_pattern = r"SCHOOL : - (\d+) (.+)"
region_pattern = r"REGION:\s+(\S+)"

# Finding the matches for 'Date', 'School code', 'School name', and 'Region' :-
date_match = re.search(date_pattern, reader)
school_match = re.search(school_pattern, reader)
region_match = re.search(region_pattern, reader)

# Giving variable name to all the data:-
date = date_match.group(1) if date_match else ""
school_code = school_match.group(1) if school_match else ""
school_name = school_match.group(2) if school_match else ""
region = region_match.group(1) if region_match else ""

# Defining the patterns to get all the outro information :-
total_candidates_pattern = r"TOTAL CANDIDATES\s*:\s*(\d+)"
total_pass_pattern = r"TOTAL PASS\s*:\s*(\d+)"
total_comptt_pattern = r"TOTAL COMPTT\.\s*:\s*(\d+)"
total_essential_repeat_pattern = r"TOTAL ESSENTIAL REPEAT\s*:\s*(\d+)"
total_absent_pattern = r"TOTAL ABSENT\s*:\s*(\d+)"

# Finding the matches of above defined pattern in the file:-
total_candidates_match = re.search(total_candidates_pattern, reader)
total_pass_match = re.search(total_pass_pattern, reader)
total_comptt_match = re.search(total_comptt_pattern, reader)
total_essential_repeat_match = re.search(total_essential_repeat_pattern, reader)
total_absent_match = re.search(total_absent_pattern, reader)

# Giving variable names to all the data :-
total_candidates = total_candidates_match.group(1) if total_candidates_match else ""
total_pass = total_pass_match.group(1) if total_pass_match else ""
total_comptt = total_comptt_match.group(1) if total_comptt_match else ""
total_essential_repeat = total_essential_repeat_match.group(1) if total_essential_repeat_match else ""
total_absent = total_absent_match.group(1) if total_absent_match else ""

In [4]:
# Pattern to match the unwanted text (from the top of the string to the line starting with "SCHOOL")
unwanted_pattern = r'DATE:.*?\n.*?-----.*?\n\nSCHOOL.*?\n'

# Remove unwanted text using re.sub
input_string_cleaned = re.sub(unwanted_pattern, '', reader, flags=re.DOTALL)

# Pattern to extract Roll, Gender, and Name
roll_gender_name_pattern = r"(\d+)\s+(\w)\s+([A-Z ]+)"

# Pattern to extract Subject Codes
subject_codes_pattern = r"(\d{3}\s+){5}(\d{3})?"

# Pattern to extract Result
result_pattern = r"\b(PASS|FAIL|COMP|ABST)\b"

# Pattern to extract Marks and Grades for each subject
marks_grades_pattern = r"(\d{3}|AB)\s+([A-Z]\d?)"

In [5]:
# Initializing an empty list to store each student's data as a dictionary
students_data = []

# Initializing variables to keep track of student information
current_student_info = None
current_student_grades = None

# Spliting the input_string by newline characters
lines = reader.strip().split('\n')

In [6]:
# Defining a Function to process and add student data to students_data list
def add_student_data(roll, gender, name, subject_codes, result, marks_grades):
    marks = []
    grades = []
    for mark_grade_tuple in marks_grades:
        mark, grade = mark_grade_tuple[0], mark_grade_tuple[1]
        if mark.isdigit():
            marks.append(int(mark))
        else:
            marks.append(mark)
        grades.append(grade)
    
    if len(subject_codes) < 6 and len(marks) < 6 and len(grades) < 6:
        subject_codes.append(np.NaN)
        marks.append(np.NaN)
        grades.append(np.NaN)

    row_data = {
        'Roll': roll,
        'Gender': gender,
        'Name': name.strip(),
        'Sub_1': subject_codes[0],
        'Marks_1': marks[0],
        'grade_1': grades[0],
        'Sub_2': subject_codes[1],
        'Marks_2': marks[1],
        'grade_2': grades[1],
        'Sub_3': subject_codes[2],
        'Marks_3': marks[2],
        'grade_3': grades[2],
        'Sub_4': subject_codes[3],
        'Marks_4': marks[3],
        'grade_4': grades[3],
        'Sub_5': subject_codes[4],
        'Marks_5': marks[4],
        'grade_5': grades[4],
        'Sub_6': subject_codes[5],
        'Marks_6': marks[5],
        'grade_6': grades[5],
        'Result': result
    }
    
    students_data.append(row_data)

In [7]:
# Iterating through each line to process student data
for line in lines:
    line = line.strip()
    # Checking if the line contains Roll, Gender, and Name
    if re.match(roll_gender_name_pattern, line):
        current_student_info = line
    # Checking if the line contains Marks and Grades
    elif re.match(marks_grades_pattern, line):
        current_student_grades = line
        # Extracting Roll, Gender, and Name
        roll, gender, name = re.search(roll_gender_name_pattern, current_student_info).groups()

        # Extracting Subject Codes and Separating individual subject codes
        subject_codes_string = re.search(subject_codes_pattern, current_student_info).group()
        subject_codes = re.findall(r"\d{3}", subject_codes_string)

        # Extracting Result
        result = re.search(result_pattern, current_student_info).group()

        # Extracting Marks and Grades for each subject
        marks_grades = re.findall(marks_grades_pattern, current_student_grades)

        # Adding student data to students_data list
        add_student_data(roll, gender, name, subject_codes, result, marks_grades)

In [8]:
# Creating the final DataFrame using the list of student dictionaries
df = pd.DataFrame(students_data)

# Reseting the index of the DataFrame
df.reset_index(drop=True, inplace=True)
df = df.fillna("")

# Printing the DataFrame
df

,Roll,Gender,Name,Sub_1,Marks_1,grade_1,Sub_2,Marks_2,grade_2,Sub_3,...,Sub_4,Marks_4,grade_4,Sub_5,Marks_5,grade_5,Sub_6,Marks_6,grade_6,Result
0,22614342,M,ABHIMANYU KUMAR,301,76,B2,302,80,B1,041,...,042,63,C1,043,52,D2,048,83,B1,PASS
1,22614343,F,AKANKSHA PRIYA,301,86,A2,302,76,B2,041,...,042,79,A2,043,74,B1,048,94,A1,PASS
2,22614344,F,AKANKSHI KUMARI,301,84,B1,322,98,A1,041,...,042,95,A1,043,95,A1,048,89,A2,PASS
3,22614345,M,AKEE KUMAR,301,54,D1,048,86,A2,041,...,042,66,B2,043,59,C2,065,73,C2,PASS
4,22614346,M,AMAN KUMAR,301,79,B2,302,74,B2,041,...,042,62,C1,043,65,C1,048,85,B1,PASS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,22614654,F,VAANI BHANWALA,301,79,B2,048,95,A1,044,...,042,68,B2,043,69,B2,,,,PASS
313,22614655,F,RASHI SINHA,301,91,A1,030,70,B2,054,...,055,54,C2,241,66,C1,802,89,A2,PASS
314,22614656,M,AAYUSH RAJ,301,AB,E,048,30,E,041,...,042,AB,E,043,AB,E,065,AB,E,ABST
315,22614657,M,KRISHI RAJ,301,AB,E,048,30,E,044,...,042,AB,E,043,AB,E,065,AB,E,ABST


In [9]:
# Function to extract numeric value from a string
def extract_numeric(value):
    numeric_part = re.sub(r'\D', '', str(value))
    return int(numeric_part) if numeric_part.isdigit() else 0

# Apply the function to extract numeric values from Marks columns
df['Marks_1'] = df['Marks_1'].apply(extract_numeric)
df['Marks_2'] = df['Marks_2'].apply(extract_numeric)
df['Marks_3'] = df['Marks_3'].apply(extract_numeric)
df['Marks_4'] = df['Marks_4'].apply(extract_numeric)
df['Marks_5'] = df['Marks_5'].apply(extract_numeric)
df['Marks_6'] = df['Marks_6'].apply(extract_numeric)

# Add a new column 'Total_Marks'
df['Total_Marks'] = df['Marks_1'] + df['Marks_2'] + df['Marks_3'] + df['Marks_4'] + df['Marks_5'] + df['Marks_6']

# Calculating and Summing Up the best five marks for each row and adding a new column 'Total Marks (Best 5)'
df['Total Marks (Best 5)'] = df[['Marks_1', 'Marks_2', 'Marks_3', 'Marks_4', 'Marks_5', 'Marks_6']].apply(lambda row: sum(sorted(row, reverse=True)[:5]), axis=1)

# Calculate percentage and add a new column 'Percentage (%)'
total_possible_marks = 500
df['Percentage (%)'] = (df['Total Marks (Best 5)'] / total_possible_marks) * 100
df['Percentage (%)'] = df['Percentage (%)'].apply(lambda x: round(x, 2))

# Printing the DataFrame
df

,Roll,Gender,Name,Sub_1,Marks_1,grade_1,Sub_2,Marks_2,grade_2,Sub_3,...,Sub_5,Marks_5,grade_5,Sub_6,Marks_6,grade_6,Result,Total_Marks,Total Marks (Best 5),Percentage (%)
0,22614342,M,ABHIMANYU KUMAR,301,76,B2,302,80,B1,041,...,043,52,D2,048,83,B1,PASS,405,354,70.8
1,22614343,F,AKANKSHA PRIYA,301,86,A2,302,76,B2,041,...,043,74,B1,048,94,A1,PASS,484,410,82.0
2,22614344,F,AKANKSHI KUMARI,301,84,B1,322,98,A1,041,...,043,95,A1,048,89,A2,PASS,556,472,94.4
3,22614345,M,AKEE KUMAR,301,54,D1,048,86,A2,041,...,043,59,C2,065,73,C2,PASS,393,339,67.8
4,22614346,M,AMAN KUMAR,301,79,B2,302,74,B2,041,...,043,65,C1,048,85,B1,PASS,418,365,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,22614654,F,VAANI BHANWALA,301,79,B2,048,95,A1,044,...,043,69,B2,,0,,PASS,374,374,74.8
313,22614655,F,RASHI SINHA,301,91,A1,030,70,B2,054,...,241,66,C1,802,89,A2,PASS,447,393,78.6
314,22614656,M,AAYUSH RAJ,301,0,E,048,30,E,041,...,043,0,E,065,0,E,ABST,49,49,9.8
315,22614657,M,KRISHI RAJ,301,0,E,048,30,E,044,...,043,0,E,065,0,E,ABST,30,30,6.0


In [10]:
# Extract unique subject codes
subject_codes = df[['Sub_1', 'Sub_2', 'Sub_3', 'Sub_4', 'Sub_5', 'Sub_6']].values.flatten()
subject_codes = np.unique(subject_codes)

# Create a new dataframe
new_columns = ['Roll', 'Gender', 'Name']
for code in subject_codes:
    new_columns.append(code)
    new_columns.append('Grade_' + code)
new_columns.append('Total Marks')                # Add a new column 'Total_Marks'
new_columns.append('Total Marks (Best 5)')         # Add a new column 'Total_Marks (Best 5)'

new_df = pd.DataFrame(columns=new_columns)

# Iterate through rows and populate the new dataframe
for _, row in df.iterrows():
    new_row = [row['Roll'], row['Gender'], row['Name']]
    total_marks = 0
    best_marks = []
    for code in subject_codes:
        matching_index = np.where(row[['Sub_1', 'Sub_2', 'Sub_3', 'Sub_4', 'Sub_5', 'Sub_6']] == code)[0]
        if matching_index.size > 0:
            marks = row['Marks_' + str(matching_index[0] + 1)]
            grade = row['grade_' + str(matching_index[0] + 1)]
            total_marks += marks           # Adding total marks in a list
            best_marks.append(marks)       # Adding total marks (Best 5) in a list
            new_row.append(marks)
            new_row.append(grade)
        else:
            new_row.append(np.NaN)
            new_row.append(np.NaN)

    # Adding all `Total_Marks` and `Total_Marks (Best 5)` in new columns
    best_marks.sort(reverse=True)
    total_marks_best_5 = sum(best_marks[:5])
    new_row.append(total_marks)
    new_row.append(total_marks_best_5)

    new_df.loc[len(new_df)] = new_row

# Calculate percentage and add a new column 'Percentage (%)'
total_possible_marks = 500
new_df['Percentage (%)'] = (new_df['Total Marks (Best 5)'] / total_possible_marks) * 100
new_df['Percentage (%)'] = new_df['Percentage (%)'].apply(lambda x: round(x, 2))

new_df = new_df.fillna('')

# Printing the new_df
new_df

,Roll,Gender,Name,,Grade_,030,Grade_030,041,Grade_041,042,...,Grade_301,302,Grade_302,322,Grade_322,802,Grade_802,Total Marks,Total Marks (Best 5),Percentage (%)
0,22614342,M,ABHIMANYU KUMAR,,,,,51.0,D1,63.0,...,B2,80.0,B1,,,,,405,354,70.8
1,22614343,F,AKANKSHA PRIYA,,,,,75.0,B1,79.0,...,A2,76.0,B2,,,,,484,410,82.0
2,22614344,F,AKANKSHI KUMARI,,,,,95.0,A1,95.0,...,B1,,,98.0,A1,,,556,472,94.4
3,22614345,M,AKEE KUMAR,,,,,55.0,C1,66.0,...,D1,,,,,,,393,339,67.8
4,22614346,M,AMAN KUMAR,,,,,53.0,C2,62.0,...,B2,74.0,B2,,,,,418,365,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,22614654,F,VAANI BHANWALA,0.0,,,,,,68.0,...,B2,,,,,,,374,374,74.8
313,22614655,F,RASHI SINHA,,,70.0,B2,,,,...,A1,,,,,89.0,A2,447,393,78.6
314,22614656,M,AAYUSH RAJ,,,,,19.0,E,0.0,...,E,,,,,,,49,49,9.8
315,22614657,M,KRISHI RAJ,,,,,,,0.0,...,E,,,,,,,30,30,6.0


In [17]:
new_df = new_df.reset_index(drop=True)
copy_df = new_df.copy()

all_classes = []
while True:

    class_data = []
    for index, row in copy_df.iterrows():
        if index == 0:
            continue  # Skip the first row since there's no previous row to compare with

        current_name = row['Name']
        previous_name = copy_df.loc[index - 1, 'Name']

        current_first_letter = current_name[0]
        previous_first_letter = previous_name[0]

        if ord(current_first_letter) < ord(previous_first_letter):
            # Add previous data to the new dataframe
            class_data.append(copy_df.loc[:index - 1])

            # Delete previous data from the original dataframe
            copy_df = copy_df.iloc[index:]
            copy_df = copy_df.reset_index(drop=True)
            break  # Exit the loop since we are deleting previous data

    # Concatenate the collected previous data into a new dataframe
    first_class_df = pd.concat(class_data, ignore_index=True)
    end_index = int(first_class_df.index[-1])

    # Iterate through columns in the first_class_df DataFrame to check for empty columns
    for col in first_class_df.columns:
        if all(first_class_df[col] == ""):
            first_class_df.drop(columns=[col], inplace=True)
    all_classes.append(first_class_df)

    if len(first_class_df) < 20:
        break

if not copy_df.empty:
    all_classes[-1] = pd.concat([all_classes[-1], copy_df], ignore_index=True)
    end_index = int(all_classes[-1].index[-1])

len(all_classes)

8

In [19]:
all_classes[0]

,Roll,Gender,Name,041,Grade_041,042,Grade_042,043,Grade_043,048,...,Grade_065,301,Grade_301,302,Grade_302,322,Grade_322,Total Marks,Total Marks (Best 5),Percentage (%)
0,22614342,M,ABHIMANYU KUMAR,51.0,D1,63.0,C1,52.0,D2,83.0,...,,76,B2,80.0,B1,,,405,354,70.8
1,22614343,F,AKANKSHA PRIYA,75.0,B1,79.0,A2,74.0,B1,94.0,...,,86,A2,76.0,B2,,,484,410,82.0
2,22614344,F,AKANKSHI KUMARI,95.0,A1,95.0,A1,95.0,A1,89.0,...,,84,B1,,,98.0,A1,556,472,94.4
3,22614345,M,AKEE KUMAR,55.0,C1,66.0,B2,59.0,C2,86.0,...,C2,54,D1,,,,,393,339,67.8
4,22614346,M,AMAN KUMAR,53.0,C2,62.0,C1,65.0,C1,85.0,...,,79,B2,74.0,B2,,,418,365,73.0
5,22614347,M,AMISH RANJAN,56.0,C1,61.0,C2,66.0,B2,85.0,...,C1,78,B2,,,,,426,370,74.0
6,22614348,M,AMRIT RAJ,59.0,C1,65.0,B2,71.0,B2,88.0,...,C1,79,B2,,,,,443,384,76.8
7,22614349,M,ANKIT KUMAR,56.0,C1,52.0,D1,33.0,E,78.0,...,D2,63,C2,,,,,335,302,60.4
8,22614350,M,ANKIT KUMAR,73.0,B1,88.0,A1,82.0,A2,96.0,...,,77,B2,89.0,A1,,,505,432,86.4
9,22614351,M,ANKIT KUMAR,62.0,B2,63.0,C1,53.0,D1,86.0,...,,72,C1,,,87.0,A1,423,370,74.0


In [20]:
all_classes[1]

,Roll,Gender,Name,041,Grade_041,042,Grade_042,043,Grade_043,048,...,Grade_065,301,Grade_301,302,Grade_302,322,Grade_322,Total Marks,Total Marks (Best 5),Percentage (%)
0,22614394,M,ADITYA GAUTAM,46.0,D1,52.0,D1,52.0,D2,88.0,...,C1,82,B1,,,,,404,358,71.6
1,22614395,M,ADITYA RAJ,27.0,E,63.0,C1,51.0,D2,98.0,...,,75,C1,60.0,D1,,,374,347,69.4
2,22614396,M,ANAND PRAKASH,58.0,C1,62.0,C1,52.0,D2,93.0,...,,70,C1,62.0,D1,,,397,345,69.0
3,22614397,M,ANANT KUMAR,85.0,A2,87.0,A1,86.0,A2,96.0,...,D1,80,B2,,,,,506,434,86.8
4,22614398,F,ANJALI YADAV,97.0,A1,96.0,A1,95.0,A1,97.0,...,,84,B1,91.0,A1,,,560,476,95.2
5,22614399,M,ANKIT KUMAR,75.0,B1,85.0,A2,73.0,B1,94.0,...,,78,B2,89.0,A1,,,494,421,84.2
6,22614400,M,ANSHU KUMAR,54.0,C2,69.0,B2,52.0,D2,83.0,...,C2,79,B2,,,,,413,361,72.2
7,22614401,M,ANURAG KUMAR,55.0,C1,63.0,C1,53.0,D1,75.0,...,C2,75,C1,,,,,399,346,69.2
8,22614402,M,ARNAV KUMAR,56.0,C1,62.0,C1,58.0,D1,64.0,...,D1,62,C2,,,,,372,316,63.2
9,22614403,M,ARYAN RAJ,53.0,C2,53.0,D1,51.0,D2,82.0,...,D1,62,C2,,,,,371,320,64.0


In [21]:
all_classes[2]

,Roll,Gender,Name,041,Grade_041,042,Grade_042,043,Grade_043,048,Grade_048,065,Grade_065,301,Grade_301,302,Grade_302,Total Marks,Total Marks (Best 5),Percentage (%)
0,22614443,F,AANCHAL KUMARI,79.0,A2,82.0,A2,93.0,A1,93.0,A1,96.0,A1,87,A2,,,530,451,90.2
1,22614444,M,AASHISH KUMAR,63.0,B2,65.0,B2,60.0,C2,86.0,A2,81.0,C1,74,C1,,,429,369,73.8
2,22614445,M,ADITYA KUMAR,62.0,B2,75.0,B1,75.0,B1,95.0,A1,92.0,B1,75,C1,,,474,412,82.4
3,22614446,M,AISHWARY KUMAR,65.0,B2,58.0,C2,74.0,B1,91.0,A2,92.0,B1,92,A1,,,472,414,82.8
4,22614447,M,ANIKET KUMAR,53.0,C2,67.0,B2,60.0,C2,90.0,A2,,,76,B2,82.0,B1,428,375,75.0
5,22614448,M,ANKIT KUMAR,67.0,B1,53.0,D1,60.0,C2,76.0,B2,75.0,C2,68,C2,,,399,346,69.2
6,22614449,M,AYUSH ANAND,91.0,A1,85.0,A2,92.0,A1,96.0,A1,96.0,A1,91,A1,,,551,466,93.2
7,22614450,M,AYUSH KUMAR,92.0,A1,85.0,A2,87.0,A2,93.0,A1,99.0,A1,92,A1,,,548,463,92.6
8,22614451,M,DURGESH KUMAR,62.0,B2,63.0,C1,72.0,B1,89.0,A2,91.0,B1,73,C1,,,450,388,77.6
9,22614452,M,GAURAV KUMAR,92.0,A1,79.0,A2,87.0,A2,93.0,A1,,,85,B1,80.0,B1,516,437,87.4


In [22]:
all_classes[3]

,Roll,Gender,Name,041,Grade_041,042,Grade_042,043,Grade_043,044,...,Grade_065,301,Grade_301,302,Grade_302,322,Grade_322,Total Marks,Total Marks (Best 5),Percentage (%)
0,22614469,M,ADITYA KUMAR,,,93.0,A1,92.0,A1,95.0,...,A2,76,B2,,,,,547,471,94.2
1,22614470,M,ADITYA KUMAR,,,52.0,D1,51.0,D2,53.0,...,,55,D1,59.0,D1,,,337,286,57.2
2,22614471,F,AIYSHA KUMARI,,,62.0,C1,60.0,C2,68.0,...,D1,81,B1,,,,,424,364,72.8
3,22614472,M,AMEET KUMAR,59.0,C1,70.0,B1,76.0,B1,79.0,...,,68,C2,,,,,439,380,76.0
4,22614473,F,ANANYA PRAKASH,,,83.0,A2,90.0,A2,91.0,...,B1,88,A2,,,,,537,454,90.8
5,22614474,F,ANSHIKA AGRAWAL,,,75.0,B1,62.0,C1,78.0,...,,79,B2,73.0,C1,,,462,400,80.0
6,22614475,F,APARNA SINGH,,,53.0,D1,53.0,D1,69.0,...,,74,C1,70.0,C1,,,394,341,68.2
7,22614476,M,AYUSH,,,52.0,D1,53.0,D1,64.0,...,D1,76,B2,,,,,387,335,67.0
8,22614477,F,AYUSHI PRIYA,,,61.0,C2,53.0,D1,60.0,...,C2,78,B2,,,,,408,355,71.0
9,22614478,F,AYUSHI SINGH,,,94.0,A1,95.0,A1,94.0,...,A2,88,A2,,,,,559,471,94.2


In [23]:
all_classes[4]

,Roll,Gender,Name,041,Grade_041,042,Grade_042,043,Grade_043,044,...,Grade_065,301,Grade_301,302,Grade_302,322,Grade_322,Total Marks,Total Marks (Best 5),Percentage (%)
0,22614511,F,AISHWARY KUMARI,,,76.0,B1,53.0,D1,73.0,...,D1,80,B2,,,,,432,379,75.8
1,22614512,M,AMIT RANJAN,54.0,C2,69.0,B2,69.0,B2,80.0,...,,73,C1,,,,,426,372,74.4
2,22614513,M,ANAND BHARTI,,,53.0,D1,53.0,D1,61.0,...,,88,A2,74.0,B2,,,414,361,72.2
3,22614514,F,ANGEL,56.0,C1,61.0,C2,59.0,C2,64.0,...,,71,C1,,,,,390,334,66.8
4,22614515,F,ANUSHKA RANI,,,95.0,A1,89.0,A2,96.0,...,,89,A2,,,99.0,A1,566,477,95.4
5,22614516,F,ARCHANA SINHA,44.0,D2,52.0,D1,52.0,D2,56.0,...,,81,B1,,,,,360,316,63.2
6,22614517,M,ARSH ARYAN,,,66.0,B2,51.0,D2,72.0,...,D1,76,B2,,,,,424,373,74.6
7,22614518,F,BAGISHA KUMARI,78.0,A2,95.0,A1,84.0,A2,94.0,...,,92,A1,,,,,539,461,92.2
8,22614519,F,C J VIDUSHEE,,,54.0,D1,60.0,C2,57.0,...,,89,A2,84.0,A2,,,419,365,73.0
9,22614520,M,DEEPAK KUMAR,68.0,B1,86.0,A2,80.0,A2,87.0,...,,94,A1,,,,,508,440,88.0


In [24]:
all_classes[5]

,Roll,Gender,Name,030,Grade_030,041,Grade_041,048,Grade_048,054,...,Grade_065,301,Grade_301,302,Grade_302,322,Grade_322,Total Marks,Total Marks (Best 5),Percentage (%)
0,22614558,F,AAKRITI,58.0,C2,54.0,C2,73.0,C1,76.0,...,,77,B2,,,,,390,338,67.6
1,22614559,M,ABHINAV SINGH,87.0,A2,53.0,C2,84.0,B1,87.0,...,,75,C1,,,,,471,418,83.6
2,22614560,M,ADITYA BHUSHAN,65.0,C1,56.0,C1,76.0,B2,68.0,...,,70,C1,,,,,400,344,68.8
3,22614561,M,ADITYA KUMAR,50.0,D1,,,64.0,C2,65.0,...,,60,D1,62.0,D1,,,357,307,61.4
4,22614562,M,ADITYA RAJ,53.0,C2,,,79.0,B2,61.0,...,C1,72,C1,,,,,397,346,69.2
5,22614563,M,AKSHAT SHREY,81.0,A2,54.0,C2,87.0,A2,93.0,...,,80,B2,,,,,487,433,86.6
6,22614564,F,ANANYA KUMARI,80.0,B1,54.0,C2,91.0,A2,88.0,...,,87,A2,,,,,485,431,86.2
7,22614565,M,ANSAL AGRAWAL,96.0,A1,47.0,D1,92.0,A1,91.0,...,,83,B1,,,,,504,457,91.4
8,22614566,F,ARPITA PARASHAR,72.0,B2,,,83.0,B1,91.0,...,C2,85,B1,,,,,468,406,81.2
9,22614567,F,ARPITASRI KASHYAP,86.0,A2,,,89.0,A2,97.0,...,C1,92,A1,,,,,532,449,89.8


In [25]:
all_classes[6]

,Roll,Gender,Name,030,Grade_030,041,Grade_041,048,Grade_048,054,...,Grade_065,301,Grade_301,302,Grade_302,322,Grade_322,Total Marks,Total Marks (Best 5),Percentage (%)
0,22614604,F,AASTHA GUPTA,90.0,A1,,,88.0,A2,83.0,...,B1,87,A2,,,,,518,439,87.8
1,22614605,F,ADITI KRISHNA,66.0,C1,,,80.0,B2,77.0,...,,78,B2,75.0,B2,,,432,376,75.2
2,22614606,M,ADITYA KUMAR,77.0,B1,,,79.0,B2,79.0,...,,79,B2,74.0,B2,,,469,395,79.0
3,22614607,M,ADITYA PRAKASH,75.0,B1,,,86.0,A2,54.0,...,C1,75,C1,,,,,421,370,74.0
4,22614608,M,AMAN RAJ,70.0,B2,,,83.0,B1,73.0,...,,86,A2,78.0,B1,,,459,390,78.0
5,22614609,F,ANANYA ANAND,87.0,A2,44.0,D2,95.0,A1,91.0,...,,88,A2,,,,,470,426,85.2
6,22614610,M,ANIMESH KUMAR,72.0,B2,,,88.0,A2,69.0,...,C1,80,B2,,,,,451,389,77.8
7,22614611,F,ANJALI YUGESHWAR,84.0,A2,45.0,D2,92.0,A1,78.0,...,,67,C2,,,,,455,410,82.0
8,22614612,F,ANJANA RANJAN,83.0,A2,,,87.0,A2,73.0,...,,83,B1,74.0,B2,,,457,400,80.0
9,22614613,M,ARMAN KUMAR,80.0,B1,47.0,D1,78.0,B2,74.0,...,,83,B1,,,,,426,379,75.8


In [26]:
all_classes[7]

,Roll,Gender,Name,,042,Grade_042,043,Grade_043,044,Grade_044,...,055,Grade_055,241,Grade_241,302,Grade_302,322,Grade_322,802,Grade_802
0,22614653,M,AADITYA KUMAR,,63.0,C1,52.0,D2,67.0,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22614654,F,VAANI BHANWALA,0.0,68.0,B2,69.0,B2,63.0,C2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22614655,F,RASHI SINHA,,,,,,,,...,54.0,C2,66.0,C1,,,,,89.0,A2
3,22614656,M,AAYUSH RAJ,,0.0,E,0.0,E,,,...,,,,,,,,,,
4,22614657,M,KRISHI RAJ,,0.0,E,0.0,E,0.0,E,...,,,,,,,,,,
5,22614658,M,SHASHIKESH KUMAR,,,,,,,,...,0.0,E,,,0.0,E,,,,


In [12]:
# # Creating an Excel Workbook
# workbook = Workbook()
# sheet = workbook.active

In [13]:
# # Adding additional information (Date, School Code, School Name, Region) to the Excel file
# sheet['A1'] = 'Date:-'
# sheet['A2'] = 'School Code'
# sheet['H1'] = 'Region:'

# sheet['B1'] = date
# sheet['B2'] = school_code
# sheet['I1'] = region

# start_column = 'C'
# end_column = 'G'
# merged_cell = sheet[start_column + '3']
# merged_cell.alignment = Alignment(horizontal='center', vertical='center')
# sheet.merge_cells(f'{start_column}3:{end_column}3')           # Merging cells for the school_name
# sheet[f'{start_column}3'] = school_name


In [14]:
# # Writing DataFrame rows to Excel, starting from the specified row
# for row in dataframe_to_rows(new_df, index=False, header=True, ):
#     sheet.append(row)
# sheet.append([' '])

# # Writing DataFrame rows to Excel, starting from the specified row
# for row in dataframe_to_rows(first_class_df, index=False, header=True, ):
#     sheet.append(row)
# sheet.append([' '])

# start_row = 4

# # Adding Outro Information :-
# end = sheet.max_row
# sheet['A'+str(end+2)] = 'Total Candidates :- '
# sheet['D'+str(end+2)] = 'Total Absent :- '
# sheet['A'+str(end+3)] = 'Total Pass :- '
# sheet['D'+str(end+3)] = 'Total Comptt. :-'
# sheet['A'+str(end+4)] = 'Total Essential Repeat :- '

# sheet['B'+str(end+2)] = total_candidates
# sheet['E'+str(end+2)] = total_absent
# sheet['B'+str(end+3)] = total_pass
# sheet['E'+str(end+3)] = total_comptt
# sheet['B'+str(end+4)] = total_essential_repeat

In [15]:
# Saving the Excel file
# workbook.save('Output_Excel.xlsx')